In [1]:
!pip install torch==2.3.0 torchvision torchaudio -f https://download.pytorch.org/whl/cu121/torch_stable.html
!pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"

Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.2 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 25.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLlamaModel
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from unsloth.models.llama import *

In [4]:
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False

In [5]:
import torch.nn as nn
from typing import Optional, List, Dict  # Assuming this is the correct import


In [6]:
class CustomEmbeddingLlamaModel(FastLanguageModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Additional embeddings initialized based on the parent's embedding dimensions
        self.input_embedding = self.get_input_embeddings().to(self.device)
        self.conversation_embedding = nn.Embedding(self.config.vocab_size, self.config.hidden_size).to(self.device)
        self.segment_embedding = nn.Embedding(self.config.vocab_size, self.config.hidden_size).to(self.device)
        self.emotion_embedding = nn.Embedding(self.config.vocab_size, self.config.hidden_size).to(self.device)
        
        # Layer normalization for the combined embeddings
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        conv_ids: Optional[torch.LongTensor] = None,
        segment_ids: Optional[torch.LongTensor] = None,
        emotion_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        # If inputs_embeds is not provided, compute it from input_ids
        if inputs_embeds is None and input_ids is not None:
            inputs_embeds = self.input_embedding(input_ids)
        
        # Add conversation, segment, and emotion embeddings if provided
        if conv_ids is not None:
            inputs_embeds += self.conversation_embedding(conv_ids)
        if segment_ids is not None:
            inputs_embeds += self.segment_embedding(segment_ids)
        if emotion_ids is not None:
            inputs_embeds += self.emotion_embedding(emotion_ids)
        
        # Normalize the combined embeddings
        inputs_embeds = self.layer_norm(inputs_embeds)
        
        # Ensure only one of input_ids or inputs_embeds is passed to the parent's forward method
#         if input_ids is not None:
#             input_ids_to_pass = input_ids
#             inputs_embeds_to_pass = None
#         else:
#             input_ids_to_pass = None
#             inputs_embeds_to_pass = inputs_embeds
        
        # Call the parent's forward method with the correct arguments
        return LlamaModel_fast_forward(
            self,
#            input_ids=input_ids_to_pass,
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            **kwargs
        )

#     def prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **kwargs):
#         # Prepare inputs for text generation, ensuring all custom embeddings are included
#         inputs = super().prepare_inputs_for_generation(input_ids, past_key_values=past_key_values, attention_mask=attention_mask, **kwargs)
        
#         # Add conversation, segment, and emotion ids to inputs if they're present in kwargs
#         for key in ['conversation_ids', 'segment_ids', 'emotion_ids']:
#             if key in kwargs:
#                 inputs[key] = kwargs[key]
        
#         return inputs

#     def _update_model_kwargs_for_generation(self, outputs, model_kwargs, is_encoder_decoder=False):
#         # Update model kwargs based on the parent's method, ensuring custom ids are carried forward
#         model_kwargs = super()._update_model_kwargs_for_generation(outputs, model_kwargs, is_encoder_decoder)
        
#         # Update conversation, segment, and emotion ids for generation
#         for key in ['conversation_ids', 'segment_ids', 'emotion_ids']:
#             if key in model_kwargs:
#                 model_kwargs[key] = model_kwargs[key][:, -1].unsqueeze(-1)
        
#         return model_kwargs


In [7]:
# Update the model initialization
model, tokenizer = CustomEmbeddingLlamaModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [8]:
model = CustomEmbeddingLlamaModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","embed_tokens", "lm_head",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


In [75]:
from datasets import load_dataset
dataset = load_dataset("DipeshChaudhary/cleanedEmpatheticDialogueInSharegptStyle", split = "train")

In [76]:
from unsloth.chat_templates import get_chat_template

In [77]:
unsloth_template = (
    "{{ bos_token }}"
    "<|start_header_id|>You are an empathetic chatbot helping user.<|end_header_id|>\n"
    "{% for message in messages %}"
        "{% if message['role'] == 'user' %}"
            "<conversation={{ loop.index }}/>\n"
            "<emotion={{ message['emotion'] }}/>\n"
            ">>> User: {{ message['content'] }}\n"
        "{% elif message['role'] == 'assistant' %}"
            "<conversation={{ loop.index }}/>\n"
            "<emotion={{ message['emotion'] }}/>\n"
            ">>> Assistant: {{ message['content'] }}\n"
            "{{ eos_token }}\n"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
        "<conversation={{ messages|length + 1 }}/>\n"
        "<therapist/>.\n"
    "{% endif %}"
)

unsloth_eos_token = "eos_token"
# 

In [78]:
tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"emotions":"emotion","role" : "from", "content" : "value", "user" : "patient", "assistant" : "therapist"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )


In [79]:
def formatting_prompts_func(examples):
    convos = examples["Conversations"]
    texts = []
    input_ids_list = []
    attention_mask_list = []
    segment_ids_list = []
    conv_ids_list = []
    emotion_ids_list = []

    for convo in convos:
        text = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        texts.append(text)
        
        # Tokenize the text
        tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
        
        input_ids = tokenized.input_ids.squeeze()
        attention_mask = tokenized.attention_mask.squeeze()
        
        # Generate segment_ids
        segment_ids = torch.zeros_like(input_ids)
        for i, message in enumerate(convo):
            start = text.index(f">>> {'User' if message['from'] == 'patient' else 'Assistant'}: {message['value']}")
            end = start + len(message['value'])
            segment_ids[start:end] = 1 if message['from'] == 'patient' else 2
        
        # Generate conv_ids
        conv_ids = torch.zeros_like(input_ids)
        for i, message in enumerate(convo):
            start = text.index(f">>> {'User' if message['from'] == 'patient' else 'Assistant'}: {message['value']}")
            end = start + len(message['value'])
            conv_ids[start:end] = i + 1
        
        # Generate emotion_ids
        emotion_ids = torch.zeros_like(input_ids)
        emotion_mapping = {
                "afraid": 1, "angry": 2, "annoyed": 3, "anticipating": 4, "anxious": 5,
                "apprehensive": 6, "ashamed": 7, "caring": 8, "confident": 9, "content": 10,
                "devastated": 11, "disappointed": 12, "disgusted": 13, "embarrassed": 14,
                "excited": 15, "faithful": 16, "furious": 17, "grateful": 18, "guilty": 19,
                "hopeful": 20, "impressed": 21, "jealous": 22, "joyful": 23, "lonely": 24,
                "nostalgic": 25, "prepared": 26, "proud": 27, "sad": 28, "sentimental": 29,
                "surprised": 30, "terrified": 31, "trusting": 32, "agreeing": 33, "acknowledging": 34,
                "encouraging": 35, "consoling": 36, "sympathizing": 37, "suggesting": 38,
                "questioning": 39, "wishing": 40, "neutral": 41}
        for i, message in enumerate(convo):
            start = text.index(f">>> {'User' if message['from'] == 'patient' else 'Assistant'}: {message['value']}")
            end = start + len(message['value'])
            emotion_ids[start:end] = emotion_mapping.get(message['emotion'], 0)
        
        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        segment_ids_list.append(segment_ids)
        conv_ids_list.append(conv_ids)
        emotion_ids_list.append(emotion_ids)

    return {
        "text": texts,
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "segment_ids": segment_ids_list,
        "conv_ids": conv_ids_list,
        "emotion_ids": emotion_ids_list,
    }        
    


In [80]:
dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/19533 [00:00<?, ? examples/s]

In [51]:
dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'segment_ids', 'conv_ids', 'emotion_ids'],
    num_rows: 19533
})

In [37]:
tokenizer

PreTrainedTokenizerFast(name_or_path='unsloth/llama-3-8b-Instruct-bnb-4bit', vocab_size=128000, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|reserved_special_token_250|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),

In [43]:
length1 =len(dataset[0]['emotion_ids'])
print(length1)

2048


In [70]:
length=len(dataset[0]['segment_ids'])
print(length)

2048


In [49]:
tokenizer.encode(dataset[0]['text'])

[128000,
 128000,
 128006,
 2675,
 527,
 459,
 36681,
 5411,
 6369,
 6465,
 10695,
 1217,
 13,
 128007,
 198,
 27,
 62737,
 28,
 16,
 10381,
 27,
 74453,
 28,
 12490,
 989,
 10381,
 20761,
 2724,
 25,
 358,
 6227,
 2133,
 311,
 1518,
 279,
 52933,
 449,
 856,
 1888,
 4333,
 13,
 1102,
 574,
 279,
 1176,
 892,
 584,
 3596,
 7543,
 892,
 7636,
 3871,
 13,
 10541,
 1070,
 574,
 264,
 2763,
 315,
 1274,
 11,
 584,
 6612,
 1093,
 279,
 1193,
 1274,
 304,
 279,
 1917,
 627,
 27,
 62737,
 28,
 17,
 10381,
 27,
 74453,
 28,
 7998,
 287,
 10381,
 20761,
 22103,
 25,
 15148,
 420,
 264,
 4333,
 499,
 1051,
 304,
 3021,
 449,
 11,
 477,
 1120,
 264,
 1888,
 4333,
 5380,
 128009,
 198,
 27,
 62737,
 28,
 18,
 10381,
 27,
 74453,
 22495,
 537,
 24823,
 292,
 10381,
 20761,
 2724,
 25,
 1115,
 574,
 264,
 1888,
 4333,
 13,
 358,
 3194,
 1077,
 627,
 27,
 62737,
 28,
 19,
 10381,
 27,
 74453,
 28,
 7998,
 287,
 10381,
 20761,
 22103,
 25,
 11208,
 706,
 1364,
 8208,
 5380,
 128009,
 198,
 27,
 62737,

# **SFTTTRAINER**

In [81]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [82]:
from transformers import DataCollatorForLanguageModeling
import torch

class CustomDataCollator(DataCollatorForLanguageModeling):
    def __call__(self, features):
        batch = super().__call__(features)
        
        # Add custom ids to the batch, if they exist
        custom_fields = ['input_ids', 'segment_ids', 'conv_ids', 'emotion_ids']
        for field in custom_fields:
            if all(field in f for f in features):
                # Convert to tensor if it's a list
                field_data = [f[field] for f in features]
                if isinstance(field_data[0], list):
                    batch[field] = torch.tensor(field_data)
                else:
                    batch[field] = torch.stack(field_data)
            else:
                print(f"Warning: '{field}' not found in all features. Skipping this field.")
        
        return batch

# Initialize the custom data collator
data_collator = CustomDataCollator(tokenizer=tokenizer, mlm=False)

In [85]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            warmup_steps = 5,
            max_steps = 60,
            learning_rate = 2e-4,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 1,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 3407,
            output_dir = "outputs",
        # ... (keep the existing arguments)
    ),
    data_collator=data_collator, 
)

max_steps is given, it will override any value given in num_train_epochs


In [86]:
# Run the training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19,533 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 1,092,616,192


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 

In [ ]:
# from torch.utils.data import Dataset
# import torch

# class PreTokenizedDataset(Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         item = self.dataset[idx]
#         return {
#             "input_ids": item["input_ids"],
#             "attention_mask": item["attention_mask"],
#             "labels": item["input_ids"],  # For language modeling, labels are typically the same as input_ids
#             "segment_ids": item["segment_ids"],
#             "conv_ids": item["conv_ids"],
#             "emotion_ids": item["emotion_ids"],
#         }

# # Create an instance of the pre-tokenized dataset
# pre_tokenized_dataset = PreTokenizedDataset(dataset)

# # Update the CustomDataCollator
# class CustomDataCollator(DataCollatorForLanguageModeling):
#     def __call__(self, features):
#         batch = super().__call__(features)
        
#         # Add custom ids to the batch
#         custom_fields = ['segment_ids', 'conv_ids', 'emotion_ids']
#         for field in custom_fields:
#             if all(field in f for f in features):
#                 batch[field] = torch.stack([f[field] for f in features])
#             else:
#                 print(f"Warning: '{field}' not found in all features. Skipping this field.")
        
#         return batch

# # Initialize the custom data collator
# data_collator = CustomDataCollator(tokenizer=tokenizer, mlm=False)

# # Update the SFTTrainer initialization
# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=pre_tokenized_dataset,
#     dataset_text_field=None,  # Set this to None since we're using a pre-tokenized dataset
#     max_seq_length=max_seq_length,
#     dataset_num_proc=2,
#     packing=False,
#     args=TrainingArguments(
#         # ... (keep existing arguments)
#     ),
#     data_collator=data_collator,
# )

# # Run the training
# trainer_stats = trainer.train()